In [2]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import time
import re


In [12]:
text_file_dir = "./data/wikispeedia_articles_plaintext/plaintext_articles/"
file_data = []

for filename in os.listdir(text_file_dir):
    if filename.endswith(".txt"):  # Ensure you're only reading .txt files
        with open(os.path.join(text_file_dir, filename), 'r', encoding='utf-8') as file:
            content = file.read()
        
            # Split the content into lines to remove the header
            lines = content.split('\n')

            # Removing the header (line 0) and join back the content
            if lines:
                title = lines.pop(2)  # Assuming the first line is the title
            
            content = ' '.join(lines).replace("\n", " ")
            
            # Create a dictionary with title and content
            file_data.append({"title": title, "text_content": content})

# Create a DataFrame
project_data = pd.DataFrame(file_data)

# Display the DataFrame structure
print(project_data.head())

article_titles = project_data['title'].tolist()
article_titles


                         title  \
0               Second Crusade   
1               Navassa Island   
2             Evan Rachel Wood   
3  Tropical Storm Henri (2003)   
4      Final Fantasy Adventure   

                                        text_content  
0     #copyright   2007 Schools Wikipedia Selecti...  
1     #copyright   2007 Schools Wikipedia Selecti...  
2     #copyright   2007 Schools Wikipedia Selecti...  
3     #copyright   2007 Schools Wikipedia Selecti...  
4     #copyright   2007 Schools Wikipedia Selecti...  


['Second Crusade',
 'Navassa Island',
 'Evan Rachel Wood',
 'Tropical Storm Henri (2003)',
 'Final Fantasy Adventure',
 'Tornado',
 'John W. Campbell',
 'Whooping Crane',
 'Shigin',
 'Hafnium',
 'Hawaiian Goose',
 'Ankylosaurus',
 'Andriyivskyy Descent',
 'Supermarine Spitfire',
 'Carl Jung',
 'Fine art',
 'Arithmetic',
 'Taj Mahal',
 'Social capital',
 'Nepal',
 'Book of Common Prayer',
 'African Buffalo',
 'Luminiferous aether',
 'Chicago',
 'Giza pyramid complex',
 'Lyme disease',
 'Humour',
 'Imperative programming',
 'Brain',
 'Coconut',
 'Tern',
 'Wars of the Roses',
 'Davis Cup',
 'Great Tit',
 'Wave–particle duality',
 'Jean Charles de Menezes',
 'Local community',
 'Music of Spain',
 'Terik',
 'Bernard Hinault',
 "Hickman's potentilla",
 'Eris (dwarf planet)',
 'Intelligence',
 'Margaret Sanger',
 'Bretton Woods system',
 'Glastonbury Festival',
 'Oxygen',
 'Gerald Ford',
 'Antarctic Treaty System',
 'John Locke',
 'Martin Luther King, Jr.',
 'Six-party talks',
 'Coot',
 'Nint

In [13]:

len(article_titles)

4604

In [11]:

def construct_url_RationalWiki(title):
    if not isinstance(title, str):
        return None  # Return None or handle the error as appropriate

    normalized_title = title.lower().replace(' ', '_')  # Convert title to lowercase and replace spaces with underscores
    return f"https://rationalwiki.org/wiki/{normalized_title}"

def construct_url_Infogalactic(title):
    if not isinstance(title, str):
        return None  # Return None or handle the error as appropriate
    normalized_title = title.lower().replace(' ', '_')  # Convert title to lowercase and replace spaces with underscores
    return f"https://infogalactic.com/info/{normalized_title}"

def construct_url_Conservapedia(title):
    normalized_title = title.lower().replace(' ', '_')  # Convert title to lowercase and replace spaces with underscores
    return f"https://www.conservapedia.com/{normalized_title}"


In [7]:
#WARNING - This cell may take a long time to run (90min) - risk of blocked IP address because of too many requests
results = []

for title in article_titles:
    url = construct_url_RationalWiki(title)
    response = requests.get(url)

    # Store the title and the response status code
    results.append({'Article Title': title, 'Response Status': response.status_code})

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Optionally, save the DataFrame to a CSV file
df.to_csv('article_status_results_RationalWiki.csv', index=False)

print(df)

                    Article Title  Response Status
0                  Second Crusade              404
1                  Navassa Island              404
2                Evan Rachel Wood              404
3     Tropical Storm Henri (2003)              404
4         Final Fantasy Adventure              404
...                           ...              ...
4599                      Réunion              404
4600                       Flower              404
4601                     Banknote              404
4602               Weyto language              404
4603                    Marseille              404

[4604 rows x 2 columns]


In [29]:
article_status_results_RationalWiki = pd.read_csv('scraped_data/article_status_results_RationalWiki.csv')
article_status_results_RationalWiki['Response Status'].value_counts()


Response Status
404    3923
200     613
503      68
Name: count, dtype: int64

In [14]:

def get_article_index(title):
    return article_titles.index(title)
print(get_article_index('Final Fantasy Adventure'))

4


In [8]:
#Test the html extract of rational wiki just for one article     

print(article_titles[1681])
url = construct_url_RationalWiki(article_titles[1681])
response = requests.get(url)
#print('Response status code: {0}\n'.format(response.status_code))
#print('Response headers: {0}\n'.format(response.headers))
#print('Response body: {0}'.format(response.text))
soup = BeautifulSoup(response.content, 'html.parser')    
paragraphs = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

for para in paragraphs:
    for edit_link in para.find_all('span', class_="mw-editsection"): 
        edit_link.decompose()
    # Extract and print the text
    if para.get_text(strip=True):
        print(para.get_text())      

Jamaica
Jamaica
Jamaica is a small island nation in the Caribbean. Along with 15 other countries, its head of state lives in a large palace in England. Jamaica is the birthplace of the Rastafari movement, a religion which posits that Haile Selassie of Ethiopia is God (or "Jah") and that enlightenment is achieved through smoking lots and lots of pot. It's also known for its really really fast sprinters, fast bowlers, excellent musicians, and bobsled team.

Homophobia
Unfortunately, it is also ranked consistently as one of the most homophobic countries in the world.[1] Male homosexual activity is illegal with a penalty of up to ten years in prison, and to make matters worse, the country has a serious problem with violent attacks on gay and transgender people. Examples include the murders of Oshane Gordon, a 16-year-old who was stabbed to death in his home in 2011, and Dwayne Jones, a 16-year-old trans woman who was killed in 2013, reportedly during a party in which somebody recognised he

In [9]:
#Create the dataframe for rationalwiki with the layout usable with the NLP file.
df = pd.read_csv('scraped_data/article_status_results_RationalWiki.csv')
df_rationalwiki_articles = df[df['Response Status'] == 200]
df_rationalwiki_articles = df_rationalwiki_articles.drop(columns=['Response Status'])
df_rationalwiki_articles = df_rationalwiki_articles.rename(columns={'Article Title': 'title'})
display(df_rationalwiki_articles)

,title
19,Nepal
22,Luminiferous aether
23,Chicago
25,Lyme disease
26,Humour
...,...
4574,Bahrain
4578,Flood
4586,Neptune
4588,Hinduism


In [9]:
#Functions to extract the plain text of the website https://rationalwiki.org/ for every page that are present
# in both RationalWiki and Wikispedia dataset

def extract_text_rationalwiki(url):
    response = requests.get(url)
    if response.status_code != 200:
        return "Error: Unable to fetch article"

    soup = BeautifulSoup(response.content, 'html.parser')

    # Remove the specific navigation box
    nav_box = soup.find('div', {'role': 'navigation', 'aria-labelledby': 'Nations_of_the_world-navigationbox'})
    if nav_box:
        nav_box.decompose()

    paragraphs = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
    
    text_content = ''
    for para in paragraphs:
        # Remove 'edit' links
        for edit_link in para.find_all('span', class_="mw-editsection"):
            edit_link.decompose()

        # Extract text
        text = para.get_text(strip=True)
        if text:
            text_content += text + '\n'
    
    return text_content

    
# Main function to update DataFrame with text content
def update_df_with_text_content(df, csv_filename):
    texts = []
    for title in df['title']:
        url = construct_url_RationalWiki(title)
        if url is not None:
            text = extract_text_rationalwiki(url)
        else:
            text = "Invalid URL or title"  # Handle the case where URL is None
        texts.append(text)
        time.sleep(1)  # Respectful crawl delay
    df['text_content'] = texts

    # Save the updated DataFrame to a CSV file
    df.to_csv(csv_filename, index=False)
    return df


In [14]:
#WARNING - This cell takes a long time to run
# Create the dataframe for rationalwiki with the plain text of articles.
df_rationalwiki_articles = update_df_with_text_content(df_rationalwiki_articles,'rationalwiki_articles.csv' )

#if you want to use the DF delete the comment on the next line :
#df_rationalwiki_articles = pd.read_csv('scraped_data/rationalwiki_articles.csv')

print(df_rationalwiki_articles) 

                   title                                       text_content
0                  Nepal  Nepal\nTheFederal Democratic Republic of Nepal...
1    Luminiferous aether  Luminiferous aether\nEther, or æther, wasthe s...
2                Chicago  Fun:Chicago\nChicago, of the oblast ofIllinois...
3           Lyme disease  Lyme disease\nLyme diseaseis an infectiousdise...
4                 Humour  Fun:Humeur\nHumeurcan refer to:\nHumeur, espec...
..                   ...                                                ...
608              Bahrain  Bahrain\nTheKingdom of Bahrainis a tiny island...
609                Flood  Global flood\nThe global floodis a (fairly sel...
610              Neptune  Neptune\nNeptunecould use some help. Please re...
611             Hinduism  Hinduism\nHinduism(not to be confused withHind...
612          Electricity  Fun:Electricity\nElectricityisa mystery from t...

[613 rows x 2 columns]


In [81]:
#Get articles titles on Infogalactic that are in our Wikispedia Dataset 
 ##WARNING - This cell may take a long time to run (90min) - risk of blocked IP address because of too many requests
results = []
for title in article_titles:
    url = construct_url_Infogalactic(title)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers, timeout = 10)
    # Store the title and the response status code
    results.append({'Article Title': title, 'Response Status': response.status_code})

# Create a DataFrame from the results
df_infogalactic_titles = pd.DataFrame(results)

# Optionally, save the DataFrame to a CSV file
df_infogalactic_titles.to_csv('infogalactic_article_status_results.csv', index=False)

print(df_infogalactic_titles)

#How many corresponding titles beetween InfoGalactic and wikispedia
df_infogalactic_titles['Response Status'].value_counts()

                    Article Title  Response Status
0                  Second Crusade              200
1                  Navassa Island              404
2                Evan Rachel Wood              404
3     Tropical Storm Henri (2003)              404
4         Final Fantasy Adventure              404
...                           ...              ...
4599                      Réunion              200
4600                       Flower              200
4601                     Banknote              200
4602               Weyto language              200
4603                    Marseille              200

[4604 rows x 2 columns]


In [15]:

#Create the dataframe for InfoGalactic with the layout for NLP file
df_infogalactic_titles = pd.read_csv('scraped_data/infogalactic_article_status_results.csv')
df_infogalactic_titles = df_infogalactic_titles[df_infogalactic_titles['Response Status'] == 200]
df_infogalactic_titles = df_infogalactic_titles.drop(columns=['Response Status'])
df_infogalactic_titles = df_infogalactic_titles.rename(columns={'Article Title': 'title'})
display(df_infogalactic_titles)

,title
0,Second Crusade
5,Tornado
7,Whooping Crane
8,Shigin
9,Hafnium
...,...
4599,Réunion
4600,Flower
4601,Banknote
4602,Weyto language


In [18]:

#Test of html extract for infogalactic 
print(article_titles[24])
url = construct_url_Infogalactic(article_titles[24])
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
response = requests.get(url, headers=headers, timeout=10)

#print('Response status code: {0}\n'.format(response.status_code))
#print('Response headers: {0}\n'.format(response.headers))
#print('Response body: {0}'.format(response.text))

# Decode response content and apply regex
html_content = response.content.decode('utf-8')
cleaned_html = re.sub(r'<templatestyles.*?>.*?</templatestyles>', '', html_content, flags=re.DOTALL)

soup = BeautifulSoup(cleaned_html, 'html.parser')

# Process paragraphs and headings
paragraphs = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

text_output = ''
for para in paragraphs:
    for edit_link in para.find_all('span', class_="mw-editsection"): 
        edit_link.decompose()
    # Extract text
    text = para.get_text(strip=True)
    if text:
        text_output += text + '\n'

print(text_output)



Giza pyramid complex
Giza pyramid complex
TheGiza pyramid complex(Arabic:أهرامات الجيزة‎‎,IPA:[ʔɑhɾɑˈmɑːt elˈɡiːzæ], "pyramidsof Giza") is anarchaeological siteon theGiza Plateau, on the outskirts ofCairo,Egypt. This complex of ancient monuments includes the threepyramidcomplexes known as the Great Pyramids, the massivesculptureknown as theGreat Sphinx, several cemeteries, a workers' village and an industrial complex. It is located in theLibyan Desert, approximately 9 km (5 mi) west of theNile riverat the old town ofGiza, and about 13 km (8 mi) southwest of Cairo city centre. The pyramids, which have historically loomed large as emblems ofancient Egypt in the Western imagination,[1][2]were popularised inHellenistictimes, when the Great Pyramid was listed byAntipater of Sidonas one of theSeven Wonders of the World. It is by far the oldest of the ancient Wonders and the only one still in existence.
Contents
The Pyramids and the Sphinx
The Pyramids of Giza consist of theGreat Pyramid of G

In [15]:
# Remove duplicates from both DataFrames (df_infogalactic_titles, df_rationalwiki_articles)
df_infogalactic_titles = df_infogalactic_titles.drop_duplicates(subset='title')
df_rationalwiki_articles = df_rationalwiki_articles.drop_duplicates(subset='title')

# Perform the merge in order to have the articles titles that appears on rationalwiki, infogalactic and wikispeedia
common_titles_df = pd.merge(df_infogalactic_titles, df_rationalwiki_articles, on='title', how='inner')

common_titles_df = common_titles_df.drop(columns=['text_content'])
# Display the DataFrames and the length of the merged DataFrame
display(common_titles_df)
print(len(common_titles_df.title.unique()))


NameError: name 'df_infogalactic_titles' is not defined

In [16]:
def extract_text_infogalactic(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    # Use the safe_request function for making the request
    response = safe_request(url, headers)
    
    if response and response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Remove templatestyles tags
        for template_style in soup.find_all("templatestyles"):
            template_style.decompose()

        text_content = ''
        paragraphs = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
        for para in paragraphs:
            # Remove 'edit' links
            for edit_link in para.find_all('span', class_="mw-editsection"):
                edit_link.decompose()
            # Extract text
            text = para.get_text(strip=True)
            if text:
                text_content += text + '\n'
        
        return text_content
    else:
        return "Error: Unable to fetch article"


    def update_df_with_infogalactic_text_content(df, csv_filename):
        texts = []
        for title in df['title']:
            url = construct_url_Infogalactic(title)
            text = extract_text_infogalactic(url)
            texts.append(text)
            time.sleep(1)  # Respectful crawl delay
        df['text_content'] = texts

        # Save the updated DataFrame to a CSV file
        df.to_csv(csv_filename, index=False)
        return df

from requests.exceptions import ConnectionError, Timeout

def safe_request(url, headers, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)
            return response
        except ConnectionError:
            time.sleep(2 ** attempt)  # Exponential backoff
        except Timeout:
            time.sleep(1)  # Simple retry after a short wait
    return None  # or raise an exception

In [28]:
##WARNING this cell might take more than 120minutes to run
#Scrap the articles on infogalactic that are both present in wikispeedia and rationalwiki
df_infogalactic_articles = update_df_with_text_content(common_titles_df, 'infogalactic_articles.csv')

#if you want to use the DF delete the comment on the next line :
#df_infogalactic_articles = pd.read_csv('scraped_data/infogalactic_articles.csv')

display(df_infogalactic_articles)

,title,text_content
0,Nepal,Nepal\nTheFederal Democratic Republic of Nepal...
1,Luminiferous aether,"Luminiferous aether\nEther, or æther, wasthe s..."
2,Chicago,"Fun:Chicago\nChicago, of the oblast ofIllinois..."
3,Lyme disease,Lyme disease\nLyme diseaseis an infectiousdise...
4,Humour,"Fun:Humeur\nHumeurcan refer to:\nHumeur, espec..."
...,...,...
603,Bahrain,Bahrain\nTheKingdom of Bahrainis a tiny island...
604,Flood,Global flood\nThe global floodis a (fairly sel...
605,Neptune,Neptune\nNeptunecould use some help. Please re...
606,Hinduism,Hinduism\nHinduism(not to be confused withHind...


In [17]:
# Function to get a random RationalWiki article title
def get_random_rationalwiki_title():
    response = requests.get('https://rationalwiki.org/wiki/Special:Random')
    soup = BeautifulSoup(response.content, 'html.parser')
    
    if soup.title and soup.title.string:
        return soup.title.string.split(' - RationalWiki')[0]
    else:
        return None

# Function to check if a title exists on InfoGalactic
def check_infogalactic(title):
    url = construct_url_Infogalactic(title)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        # Check if response is successful and title is in the response text
        is_successful = response.status_code == 200 and title in response.text
        return is_successful, is_successful
    except requests.exceptions.RequestException as e:
        return False, str(e)


In [2]:
#WARNING - This cell takes a long time to run - It finds 3000 articles that are in both RationalWiki and InfoGalactic but not in wikispeedia 
#this will allow us to train our model.

# Initialize an empty DataFrame
training_articles2 = pd.read_csv('training_articles2.csv')

# Loop until collection of 3000 unique articles
while len(training_articles2) < 3000:
    random_title = get_random_rationalwiki_title()

    # Check conditions
    success, message = check_infogalactic(random_title)
    if random_title not in article_titles and random_title not in training_articles2['Title'].values and success:
        # Add to DataFrame if conditions are met
        training_articles2.loc[len(training_articles2)] = [random_title]

    # Save to CSV file after every 100 additions
    if len(training_articles2) % 100 == 0:
        training_articles2.to_csv('training_articles2.csv', index=False)

print("Collection complete. Final list saved to 'training_articles2.csv'.")


'\n# Initialize an empty DataFrame\ntraining_articles2 = pd.read_csv(\'training_articles2.csv\')\n\n# Loop until collection of 3000 unique articles\nwhile len(training_articles2) < 3000:\n    random_title = get_random_rationalwiki_title()\n\n    # Check conditions\n    success, message = check_infogalactic(random_title)\n    if random_title not in article_titles and random_title not in training_articles2[\'Title\'].values and success:\n        # Add to DataFrame if conditions are met\n        training_articles2.loc[len(training_articles2)] = [random_title]\n\n    # Save to CSV file after every 100 additions\n    if len(training_articles2) % 100 == 0:\n        training_articles2.to_csv(\'training_articles2.csv\', index=False)\n\nprint("Collection complete. Final list saved to \'training_articles2.csv\'.")'

In [18]:
training_articles = pd.read_csv('training_articles2.csv')
training_articles


,Title
0,Electroconvulsive therapy
1,Juche
2,Special pleading
3,Chrislam
4,Black box
...,...
2995,Four-term fallacy
2996,Nosode
2997,Very special episode
2998,The Man in the High Castle


In [19]:
training_articles = training_articles.rename(columns={'Title': 'title'})
df_infogalactic_articles = update_df_with_text_content(training_articles, '3000_infogalactic_articles.csv')